# **Part-C**


### Imports for Part C: Caption Source Classification

This section sets up the environment for building a classifier to distinguish between captions generated by different models (e.g., `custom` vs `smolvlm`).

1. **Core Libraries**:
   - `os`, `pandas`: File handling and data manipulation.
   - `torch`, `nn`, `DataLoader`, `Dataset`: PyTorch tools for defining and training the classification model.

2. **Transformers**:
   - `BertTokenizer`, `BertModel`: Tokenizer and model components from HuggingFace's BERT.
   - `get_linear_schedule_with_warmup`: Learning rate scheduler for stable training.

3. **Optimization**:
   - `AdamW`: Weight-decay variant of Adam optimizer from `torch.optim`.

4. **Evaluation & Splitting**:
   - `train_test_split` from scikit-learn to split the dataset.
   - `accuracy_score`, `classification_report` for performance evaluation.

These tools are essential for implementing a BERT-based classifier to identify the source of a caption based on its textual content.


In [26]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW  # ← AdamW is now from torch.optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

### Load and Prepare Caption Classification Data

This function preprocesses the caption dataset for training a classifier that distinguishes between captions generated by different models.

1. **Read CSV File**:
   - Loads the dataset from a CSV file using `pandas`.

2. **Clean the Data**:
   - Removes rows with missing values in critical columns such as:
     - `original_caption`
     - `generated_caption`
     - `perturbation_level`
     - `model`
     - `filename`

3. **Text Formatting**:
   - Ensures captions and perturbation levels are treated as strings.
   - Concatenates `original_caption`, `generated_caption`, and `perturbation_level` into a single string using `<SEP>` as a delimiter.
   - This combined string serves as the input for BERT.

4. **Label Encoding**:
   - Creates a binary label:
     - `0` for `smolvlm`
     - `1` for `custom` (or other models)

5. **Return**:
   - A cleaned and formatted DataFrame with three columns:
     - `filename`: Image file name (for tracking)
     - `input_text`: Combined input for classification
     - `label`: Ground truth label indicating the source model

This function is essential for preparing the dataset for a BERT-based binary classification task in Part C.


In [27]:
def load_caption_data(data_file):
    df = pd.read_csv(data_file)

    # Drop rows with missing values in relevant columns
    df = df.dropna(subset=["original_caption", "generated_caption", "perturbation_level", "model", "filename"])
    
    # Ensure all are strings and format input text
    df["original_caption"] = df["original_caption"].astype(str)
    df["generated_caption"] = df["generated_caption"].astype(str)
    df["perturbation_level"] = df["perturbation_level"].astype(str)

    df["input_text"] = df["original_caption"] + " <SEP> " + df["generated_caption"] + " <SEP> " + df["perturbation_level"]

    # Create binary labels
    df["label"] = df["model"].apply(lambda x: 0 if x.lower() == "smolvlm" else 1)

    return df[["filename", "input_text", "label"]]


### CaptionClassificationDataset: Custom Dataset for BERT Classifier

This class defines a custom PyTorch `Dataset` for training a BERT-based classifier on caption comparison data.

1. **Initialization (`__init__`)**:
   - Accepts a list of input texts, corresponding labels, a tokenizer, and a maximum sequence length.
   - Stores these for use in batching and tokenization.

2. **Length (`__len__`)**:
   - Returns the number of samples in the dataset.

3. **Item Access (`__getitem__`)**:
   - Tokenizes the input text at the given index.
   - Applies truncation and padding to a fixed maximum length (`max_length`).
   - Returns a dictionary with:
     - `input_ids`: Encoded token IDs.
     - `attention_mask`: Mask for non-padding tokens.
     - `label`: Binary label as a PyTorch tensor.

This dataset is designed to be used with a `DataLoader` for efficient batching during training and evaluation of the BERT classifier in Part C.


In [28]:
import torch
from torch.utils.data import Dataset

class CaptionClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            return_tensors="pt",
            max_length=self.max_length,
            padding="max_length",
            truncation=True
        )
        return {
            'input_ids': encoding["input_ids"].squeeze(),
            'attention_mask': encoding["attention_mask"].squeeze(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }


### CaptionClassifier: BERT-Based Binary Classification Model

This class defines a neural network for classifying whether a given caption pair was generated by SmolVLM or a custom model.

1. **Initialization (`__init__`)**:
   - Loads a pretrained BERT model using the specified model name (e.g., `bert-base-uncased`).
   - Adds a dropout layer (`0.1` probability) to prevent overfitting.
   - Includes a fully connected layer (`nn.Linear`) that maps the BERT output to the desired number of classes (`num_classes`, typically 2).

2. **Forward Pass (`forward`)**:
   - Takes `input_ids` and `attention_mask` as input.
   - Extracts the pooled output (`[CLS]` token representation) from the BERT model.
   - Applies dropout and feeds the result through the linear layer to produce logits for classification.

This model is the core of the binary classifier used in Part C to determine which model generated a given caption.


In [ ]:
class CaptionClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(CaptionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):  # ← make sure this is indented correctly
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits


### train_classifier: Training Loop for the BERT Classifier

This function performs one training epoch over the classification dataset.

1. **Model in Training Mode**:
   - Sets the model to training mode using `model.train()` to enable dropout and gradient tracking.

2. **Batch Iteration**:
   - Iterates through the DataLoader, processing one batch at a time.

3. **Gradient Reset**:
   - Clears old gradients with `optimizer.zero_grad()`.

4. **Move Data to Device**:
   - Transfers input IDs, attention masks, and labels to the specified device (CPU or GPU).

5. **Forward Pass**:
   - Computes logits by passing the inputs through the model.

6. **Loss Calculation**:
   - Uses `CrossEntropyLoss` to compute the classification loss between predicted logits and ground truth labels.

7. **Backward Pass and Optimization**:
   - Performs backpropagation with `loss.backward()`.
   - Updates model weights using the optimizer.
   - Steps the learning rate scheduler.

This function is part of the training loop used to fine-tune the BERT classifier in Part C.


In [ ]:
def train_classifier(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

### evaluate_classifier: Evaluation Function for the BERT Classifier

This function evaluates the performance of the classifier on a given dataset.

1. **Evaluation Mode**:
   - Sets the model to evaluation mode with `model.eval()` to disable dropout and gradient computation.

2. **Batch Iteration (No Gradients)**:
   - Loops through each batch in the DataLoader using `torch.no_grad()` to prevent gradient calculations.

3. **Move Data to Device**:
   - Sends inputs and labels to the appropriate device (CPU or GPU).

4. **Prediction**:
   - Computes logits with the model and uses `torch.max` to get the predicted class labels.

5. **Collect Predictions and Labels**:
   - Aggregates the predictions and actual labels for the entire dataset.

6. **Compute Metrics**:
   - Uses `classification_report` from `sklearn` to calculate precision, recall, F1-score, etc.
   - Returns both the accuracy score and the full classification report as a dictionary.

This function is used in Part C to evaluate how well the classifier can distinguish between captions generated by SmolVLM and the custom model.


In [ ]:
from sklearn.metrics import classification_report

def evaluate_classifier(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    
    # Get report as dict for macro scores
    report_dict = classification_report(actual_labels, predictions, output_dict=True)
    return accuracy_score(actual_labels, predictions), report_dict


In [ ]:
### predict_model_source: Predicts the Model Source Based on Captions

This function takes an original caption, a generated caption, and a perturbation level to predict whether the caption was generated by SmolVLM or a custom model.

1. **Evaluation Mode**:
   - Sets the model to evaluation mode using `model.eval()` to ensure no gradients are computed during prediction.

2. **Input Construction**:
   - Constructs the input text by concatenating the original caption, generated caption, and perturbation level with a separator (`<SEP>`).

3. **Tokenization**:
   - Tokenizes the input text using the provided tokenizer with padding and truncation to ensure consistent input length.

4. **Prediction**:
   - Passes the tokenized inputs through the model to obtain logits.
   - Applies `torch.max` to extract the predicted class (0 for SmolVLM, 1 for Custom).

5. **Interpretation**:
   - Interprets the model's prediction, returning a label for the corresponding model source:
     - `"Model A (SmolVLM)"` if the prediction is 0.
     - `"Model B (Custom)"` if the prediction is 1.

This function is used in Part C to classify whether the generated caption comes from SmolVLM or the custom model based on the provided captions and perturbation levels.


In [32]:
def predict_model_source(original_caption, generated_caption, perturbation_level, model, tokenizer, device, max_length=128):
    model.eval()
    
    # Construct input text in the required format
    input_text = f"{original_caption} <SEP> {generated_caption} <SEP> {perturbation_level}"
    
    # Tokenize
    encoding = tokenizer(input_text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    
    # Interpret prediction
    return "Model A (SmolVLM)" if preds.item() == 0 else "Model B (Custom)"


### Parameter Setup

The following parameters are configured for training the BERT-based caption classifier:

1. **BERT Model Name**:
   - `bert-base-uncased`: The BERT model used for encoding the input text. It is pre-trained on a large corpus of text and is case-insensitive (`uncased`).

2. **Number of Classes**:
   - `2`: The model will classify captions into two categories: "SmolVLM" and "Custom."

3. **Maximum Input Length**:
   - `128`: The maximum length of the input sequence after tokenization. Longer sequences will be truncated, and shorter ones will be padded.

4. **Batch Size**:
   - `16`: The number of samples processed together in each forward/backward pass.

5. **Number of Epochs**:
   - `4`: The number of times the entire dataset will be passed through the model during training.

6. **Learning Rate**:
   - `2e-5`: The learning rate for the optimizer. It controls how quickly the model updates its parameters during training.

These parameters are set to ensure efficient training while preventing overfitting and underfitting.


In [33]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

### split_caption_data_by_filename: Split the DataFrame into Train, Validation, and Test Sets

This function splits the input DataFrame into three sets: training, validation, and testing. The split is done based on unique filenames to ensure no overlap between the sets.

1. **Input Parameters**:
   - `df`: The DataFrame containing the data with a column `filename` for the image filenames.
   - `train_ratio`: Proportion of data used for training (default is 0.7).
   - `val_ratio`: Proportion of data used for validation (default is 0.1).
   - `test_ratio`: Proportion of data used for testing (default is 0.2).
   - `random_state`: Seed for random splitting (default is 42).

2. **Assertions**:
   - Checks that the sum of the train, validation, and test ratios is exactly 1.0.

3. **Step 1: Train vs Temp Split**:
   - Splits the unique filenames into `train_filenames` (for training) and `temp_filenames` (for validation and testing combined).

4. **Step 2: Val vs Test Split**:
   - Further splits the `temp_filenames` into `val_filenames` and `test_filenames` based on the specified validation and testing ratios.

5. **Step 3: Filter the DataFrame**:
   - Filters the main DataFrame to create separate DataFrames for training, validation, and testing by matching the filenames.

6. **Returns**:
   - The function returns three DataFrames: `train_df`, `val_df`, and `test_df`.

This function is helpful for preparing the dataset into distinct sets for model training and evaluation, ensuring no overlap between them.


In [34]:
def split_caption_data_by_filename(df, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2, random_state=42):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-5, "Ratios must sum to 1.0"
    
    # Unique filenames
    unique_filenames = df['filename'].unique()
    
    # First split: Train vs Temp (Val + Test)
    train_filenames, temp_filenames = train_test_split(
        unique_filenames,
        test_size=(1 - train_ratio),
        random_state=random_state
    )
    
    # Second split: Val vs Test
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_filenames, test_filenames = train_test_split(
        temp_filenames,
        test_size=(1 - val_ratio_adjusted),
        random_state=random_state
    )

    # Now filter the main DataFrame by filenames
    train_df = df[df['filename'].isin(train_filenames)]
    val_df = df[df['filename'].isin(val_filenames)]
    test_df = df[df['filename'].isin(test_filenames)]

    return train_df, val_df, test_df


### Loading and Splitting the Data

This section demonstrates how to load the caption data and split it into training, validation, and test sets for model evaluation.

1. **Loading the Data**:
   - The data is loaded from a CSV file (`occlusion_eval_partC.csv`) using the `load_caption_data` function, which processes the file and prepares it for training.

2. **Splitting the Data**:
   - The `split_caption_data_by_filename` function is used to split the dataset into three sets:
     - **Training Set**: 70% of the data.
     - **Validation Set**: 10% of the data.
     - **Test Set**: 20% of the data.
   - The split is done based on unique filenames to ensure no overlap between the sets.

3. **Extracting Input Texts and Labels**:
   - From the split DataFrames (`train_df`, `val_df`, and `test_df`), the input text (`input_text`) and corresponding labels (`label`) are extracted and stored in separate lists:
     - `train_texts`, `train_labels`
     - `val_texts`, `val_labels`
     - `test_texts`, `test_labels`
   
This preparation is necessary before training or evaluating a model, ensuring the data is cleanly partitioned for proper validation.


In [35]:
# Load your data
data_file = "/kaggle/input/datacsv/occlusion_eval_partC.csv"
df = load_caption_data(data_file)

# Split by image filenames
train_df, val_df, test_df = split_caption_data_by_filename(df)

# Extract input_text and labels for each split
train_texts = train_df["input_text"].tolist()
train_labels = train_df["label"].tolist()

val_texts = val_df["input_text"].tolist()
val_labels = val_df["label"].tolist()

test_texts = test_df["input_text"].tolist()
test_labels = test_df["label"].tolist()


### Model Training and Evaluation

This section covers the setup, training, and evaluation of a BERT-based classifier for distinguishing between captions generated by SmolVLM and a custom model.

1. **Tokenizer Initialization**:
   - A BERT tokenizer is initialized using the `BertTokenizer.from_pretrained` method to preprocess the input text for the classifier.

2. **Dataset Creation**:
   - The `CaptionClassificationDataset` class is used to create datasets for the training and validation splits. This class handles tokenization and transformation of the input text into the required format for BERT.

3. **DataLoader Creation**:
   - DataLoaders are created for both training and validation datasets, allowing efficient batching and shuffling during training.

4. **Device and Model Setup**:
   - The model is moved to the appropriate device (`cuda` if available, otherwise `cpu`). The `CaptionClassifier` model is instantiated with a BERT backbone, configured for binary classification.

5. **Optimizer and Scheduler**:
   - The optimizer is set to `AdamW`, and a learning rate scheduler is initialized with linear warmup, ensuring a smooth training process.

6. **Training Loop**:
   - The training loop iterates over multiple epochs, training the classifier on the training dataset. After each epoch, the model is evaluated on the validation dataset, and accuracy and detailed classification reports are printed.

7. **Model Saving**:
   - After training, the model’s state_dict is saved to a file (`bert_caption_classifier.pth`) for later use.

This procedure ensures that the model is trained to classify captions generated by different models and can be evaluated on validation data for performance.


In [ ]:
from transformers import BertTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.optim import AdamW 

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# Create datasets
train_dataset = CaptionClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CaptionClassificationDataset(val_texts, val_labels, tokenizer, max_length)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Set up device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CaptionClassifier(bert_model_name, num_classes).to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_classifier(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate_classifier(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

# Save the model
torch.save(model.state_dict(), "bert_caption_classifier.pth")


Epoch 1/4
Validation Accuracy: 0.9762
{'0': {'precision': 0.9550173010380623, 'recall': 1.0, 'f1-score': 0.9769911504424779, 'support': 276}, '1': {'precision': 1.0, 'recall': 0.9518518518518518, 'f1-score': 0.9753320683111955, 'support': 270}, 'accuracy': 0.9761904761904762, 'macro avg': {'precision': 0.9775086505190311, 'recall': 0.9759259259259259, 'f1-score': 0.9761616093768366, 'support': 546}, 'weighted avg': {'precision': 0.9772614928324272, 'recall': 0.9761904761904762, 'f1-score': 0.9761707252127227, 'support': 546}}
Epoch 2/4
Validation Accuracy: 0.9762
{'0': {'precision': 0.9550173010380623, 'recall': 1.0, 'f1-score': 0.9769911504424779, 'support': 276}, '1': {'precision': 1.0, 'recall': 0.9518518518518518, 'f1-score': 0.9753320683111955, 'support': 270}, 'accuracy': 0.9761904761904762, 'macro avg': {'precision': 0.9775086505190311, 'recall': 0.9759259259259259, 'f1-score': 0.9761616093768366, 'support': 546}, 'weighted avg': {'precision': 0.9772614928324272, 'recall': 0.976

### Model Evaluation on Test Set

This section evaluates the trained BERT classifier on the test set, reporting the accuracy and detailed classification metrics.

1. **Test Dataset Setup**:
   - The test dataset is created using the `CaptionClassificationDataset` class, where the `test_texts` and `test_labels` are tokenized and formatted for input into the BERT model.
   
2. **Test DataLoader Creation**:
   - A `DataLoader` is initialized for the test dataset, allowing the model to efficiently process the data in batches.

3. **Test Evaluation**:
   - The model is evaluated on the test dataset using the `evaluate_classifier` function, which computes the accuracy and generates a detailed classification report. This function returns the accuracy score along with the full classification report, which includes precision, recall, and F1-score metrics for each class.

4. **Results Display**:
   - The test accuracy and classification report are printed for analysis, allowing for an assessment of how well the classifier generalizes to unseen data.



In [ ]:
from sklearn.metrics import classification_report

# Prepare test set
test_dataset = CaptionClassificationDataset(test_texts, test_labels, tokenizer, max_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# evaluate_classifier on test set
test_accuracy, test_report = evaluate_classifier(model, test_dataloader, device)
print(f"Test Accuracy: {test_accuracy:.4f}")
print("Classification Report on Test Set:")
print(test_report)


Test Accuracy: 0.9700
Classification Report on Test Set:
{'0': {'precision': 0.9615384615384616, 'recall': 0.9803921568627451, 'f1-score': 0.970873786407767, 'support': 561}, '1': {'precision': 0.9792060491493384, 'recall': 0.9592592592592593, 'f1-score': 0.9691300280636108, 'support': 540}, 'accuracy': 0.9700272479564033, 'macro avg': {'precision': 0.9703722553439, 'recall': 0.9698257080610022, 'f1-score': 0.9700019072356889, 'support': 1101}, 'weighted avg': {'precision': 0.9702037633639597, 'recall': 0.9700272479564033, 'f1-score': 0.9700185370836576, 'support': 1101}}


### Macro Average Metrics

The following macro average metrics are derived from the classification report on the test set:

- **Macro Precision**: Measures the average precision across all classes, treating all classes equally.
- **Macro Recall**: Measures the average recall across all classes, treating all classes equally.
- **Macro F1-Score**: The average F1-score across all classes, which balances precision and recall.



In [38]:
print("Macro Precision:", test_report["macro avg"]["precision"])
print("Macro Recall:", test_report["macro avg"]["recall"])
print("Macro F1-score:", test_report["macro avg"]["f1-score"])


Macro Precision: 0.9703722553439
Macro Recall: 0.9698257080610022
Macro F1-score: 0.9700019072356889
